# R Package Dev

_A walkthrough in R package development_ by Spencer Seale

Last revised October 2020

<br>

### Why you should want to make an R Package

R by itself is a useful programming language. Out of the box it comes packed with functions developed for data manipulation and statistical analysis that users take advantage of regularly. However, what makes R especially powerful is the framework it provides for allowing users to openly create and distribute custom packages. Because of this, thousands of open-source R packages are available for just about any common data science task. But packages can contain functions for anything, seriously anything, check this link out if you need to get your creative juices flowing: https://rpubs.com/dbecker7/StrangeRThings

Installing packages specific to your R-related work is likely something you already do, but you may not know how easy making your own package actually is. By completing this tutorial, you'll be able to quickly build your own package from the ground up and be ushered into the world of package creation, quickly forgetting what `source()` even does.

<br>

### Boring, but interesting knowledge critical to package development

Making R packages is __cool__. But there are still some less exciting things that we should touch on prior to creating our first package. 

__Don't break the environment__

An R package is a self-contained ecosystem that should not alter a user's environment. What this means is that when a person runs: `library(your_cool_package)` their search path should not be updated with anything but the package itself (i.e. no other packages are added). This is important as whenever packages are added to the search path they bring in new objects and if those objects mask something currently in the envrionment they may uninentionally disrupt the environment. Basically, if whenever your package is attached and it causes `ggplot2` to also be added you're changing their envrionment likely without their knowledge -- don't do that!

To explore this further, spin up a fresh R session and run `search()` to view the search path. There should just be the base packages that whatever current R version you're using includes. As you attach packages to your current session, those packages will be added to the search path, making the functions and data included in them available to the user without having to reference the package individually.

To see all the objects in a particular package, run: `ls("package:a_cool_package")`. Remember, the search path in R works the same as $PATH, its what R looks to whenver a function or data object is referenced. Interestingly, `.GlobalEnv` is listed first and will always be checked before looking through packages for a function call, something to keep in mind.

__Load don't attach dependencies__

What about using other packages within your own package? Of course this is possible, but there is a right and wrong way to do this based on what was just discussed. 

* Loading makes objects available without modifying the search path
    * `a_cool_package::a_cool_function()` is how to load a function. Here, the package name is directly referenced and R knows exactly what package to look within for a specific function and does not check the search path. Additionally, this does not add `a_cool_package` to the search path, thus not altering the envrionment.


* Attaching adds all of the contents in a package's NAMESPACE to R so that functions may be used without direct referencing with `::`. This allows calling functions without their package name, but this may modify how R checks for other functions and should not be done when building R packages. 

__Anytime__ objects from another package are used in your function code, load them and never use `source()` or `library()` in your package functions. This will ensure your package does not alter the user's search path.

<br>

### Step-by-step guide to package creation 

#### 1. Installing required dev packages

It would only make sense that someone thought to use their package development skills to build a package for building packages. Thankfully, this has made the process much more tolerable and automated all of the boring tasks so that we can concentrate on the important stuff: writing functions. 

`devtools` is the current kingpin for R package development and is what we'll be using to build our package.

In [ ]:
# attach devtools
if (!"devtools" %in% installed.packages()) install.packages("devtools")
library(devtools)

Additionally, we want to install `roxygen2`. This package works in tandem with devtools and allows user-friendly annotation of package functions. This package allows us to create the function manuals directly above functions.

In [ ]:
# attach roxygen2
if (!"roxygen2" %in% installed.packages()) install.packages("roxygen2")
library(roxygen2)

#### 2. Initialize your package directory

Now that we have the necessary package-building packages, we can initialize a directory containing all the necessary components for your package including: NAMESPACE, DESCRIPTION, R.

* __NAMESPACE__ is a file that lists all of the exported and imported functions. This basically tells the package what functions to make available to the user.

* __DESCRIPTION__ is a file containing author name, version name, title, description, dependencies, and more.

* __R__ is the directory where all package R scripts will be contained. Each function should exist as an individual .R file. This will make updating individual functions easier by keeping them each within their own files and thus reducing the chances of affecting other functions during the inevitable debugging sessions. 

This is done with `create()` which adds the initial components to the file location of your choice.

In [ ]:
# example: ~/mypackage
pkg_dir <- readline(prompt = "Input file path to your new package directory: ")
if (!substr(pkg_dir, length(pkg_dir), length(pkg_dir)) == "/") pkg_dir <- paste0(pkg_dir, "/")

# initializing dir
create(pkg_dir)

In [ ]:
# view the contents
system(paste0("ls ", pkg_dir), intern = T)

#### 3. Adding a function

Now that we have the skeleton for our package, we can start adding things to it. Let's make a simple function and then export it as a .R script to `R/`.

Above the function, we need to insert roxygen2 lines using `#'`. In these lines we insert tags beginning with `@` that allow you to specify the various parts of the function documentation that are viewable when pulling up a help page. After all of the roxygen2 tags are added, we then run `document()` which creates .Rd files (stored in `man/`) for all function scripts within `R/` and adds the exported functions to the the NAMESPACE.

There are many different roxygen2 tags to add (https://cran.r-project.org/web/packages/roxygen2/vignettes/rd.html), but I'll be sticking to the critical ones I believe are necessary for the purposes of this tutorial. 

* `#' @title` input the title of the function. This will be in large font and easily viewable at the top of the help page.
* `#' @description` briefly insert what the function does.
* `#' @param` insert all the arguments for the function. Use a separate tag for each individual argument.
* `#' @details` here is a good place to add any important details the user might need for running the function. Especially if the function is fairly complex it's helpful to include this tag.
* `#' @return` identify the output. A data frame, vector? State that here.
* `#' @author` writer of the function.
* `#' @examples` add some examples for running the function. It's especially helpful if there are man different combinations of arguments that may be used to illustrate this more clearly to users. 
* `#' @export` This is an important tag. Write the name of the object you'd like to export to the NAMESPACE, which in this case, is the name of the function.


* Along with tags, `\{}` can be used to specify formatting, some examples are included in the function block below.

Look at how and where these tags are added to our example function below.


In [ ]:
#' @title Random Number
#'
#' @description Input a minimum and maximum number to generate a random number within that interval
#'
#' @param \code{minimum} Insert the minimum int for the interval
#'
#' @param \code{maximum} Insert the maximum int for the interval
#'
#' @details Set custom minimum and maximum ints to pull a random number from. Numbers must be int and both arguments are required
#'
#' @return An int between the set minimum and maximum
#'
#' @author Henrietta Lacks
#'
#' @examples
#' \dontrun{
#' random_number(5,20)
#' }
#'
#' @export random_number

random_number <- function(minimum=NULL, maximum=NULL) {
    if (!is.null(minimum) && !is.null(maximum)) {
        out <- sample(minimum:maximum, 1)
        return(out)
    } else {
        stop("One or both inputs missing.")
    }
}

In [ ]:
# testing it
tst <- random_number(5, 10)
stopifnot(tst >= 5 || tst <= 10)

* Our function is created and contains the appropriate tags, save it as an .R script to the `R/` dir in our package.

In [ ]:
# writing above code out to a .R file
writeLines("#' @title Random Number
#'
#' @description Input a minimum and maximum number to generate a random number within that interval
#'
#' @param \\code{minimum} Insert the minimum int for the interval
#'
#' @param \\code{maximum} Insert the maximum int for the interval
#'
#' @details Set custom minimum and maximum ints to pull a random number from. Numbers must be int and both arguments are required
#'
#' @return An int between the set minimum and maximum
#'
#' @author Henrietta Lacks
#'
#' @examples
#' \\dontrun{
#' random_number(5,20)
#' }
#'
#' @export random_number
random_number <- function(minimum=NULL, maximum=NULL) {
    if (!is.null(minimum) && !is.null(maximum)) {
        out <- sample(minimum:maximum, 1)
        return(out)
    } else {
        stop(\"One or both inputs missing.\")
    }
}",
paste0(pkg_dir, "/R/random_number.R"))

In [ ]:
# view the contents of R/
system(paste0("ls ", pkg_dir, "/R"), intern = T)

#### 4. Package documenting and installation

Next, we run `document()` to export the function to the `NAMESPACE` (making it available to users) and create .Rd files. The .Rd files are added into `man/` a new dir that will be automatically created after running this function. Once running the below code, take some time to look through the updated `NAMESPACE` and `man/`. `document()` must be run any time man of a function (i.e. the roxygen2 tags) or data objects are added. Additionally, its always safe after making changes to various functions to run `document()` prior to a fresh install so don't be afraid to run it each time your reinstall.

In [ ]:
document(pkg_dir)

There should be a conflict when `document()` was run and this is a great learning experience. Because we defined the function `random_number()` in the global environmnet of this tutorial and also added a new function to our package with the same name, there is a conflict in which function takes priority. The function defined in the global envrionment will mask functions having the same name that are attached as part of a package. The only way to prioritize the package function is to load it directly each time it is called using `::` notation. To prove this point, we can remove the local function and re-run `document()` and no errors should be thrown.

In [ ]:
rm("random_number")
document(pkg_dir)

If you'd like, you can edit the DESCRIPTION file by adding items such as author and version for example. Now that our function is documented and the NAMESPACE is prepared, we can install the package to our machine. 

In [ ]:
install(pkg_dir)

In [ ]:
# attach our package
library(basename(pkg_dir), character.only = T)

# open help page and see how our tags manifested
?random_number

# verify the function works
random_number(10, 55)

The package is now installed on our machine and we can attach and/or load it whenever we want. Each time functions are updated, we must re-run `document()` and re-install. You should also restart your R envrionment after each installation so that the updates are added (not necessary to do in this tutorial). Additionally `load_all()` can be used to simulate installation and subsequent attachment without actually installing the package onto the hard drive. This is beneficial for developing functions so that you can make changes then quickly attach the new version to test it out. 


One thing to note. When building a package function, if you need to call another function also created in the same package, there is no need to `source()` or load it with `::`, just call the function like its already attached.

#### 5. Adding external packages to your functions

We successfully created a function, but it's pretty boring and only included `base` functions. Now lets modify our function so that it includes a function from the `stringr` package.

In [ ]:
# overwriting random_number.R with updated code

writeLines("#' @title Random Number
#'
#' @description Input a minimum and maximum number to generate a random number within that interval
#'
#' @param \\code{minimum} Insert the minimum int for the interval
#'
#' @param \\code{maximum} Insert the maximum int for the interval
#'
#' @details Set custom minimum and maximum ints to pull a random number from. Numbers must be int and both arguments are required
#'
#' @return An int between the set minimum and maximum
#'
#' @author Henrietta Lacks
#'
#' @examples
#' \\dontrun{
#' random_number(5,20)
#' }
#'
#' @export random_number
random_number <- function(minimum=NULL, maximum=NULL) {
    if (!is.null(minimum) && !is.null(maximum)) {
        out <- sample(minimum:maximum, 1)
        print(stringr::str_detect(out, \"[0-9]+\"))
        return(out)
    } else {
        stop(\"One or both inputs missing.\")
    }
    
}",
paste0(pkg_dir, "/R/random_number.R"))

To ensure that users of our package also have the `stringr` package, we need to list it as an import to the DESCRIPTION file. We do this by adding a line to the DESCRIPTION file: `Imports:`. Underneath it, we list each package that is referenced in our package, 1 line per package separated by commas. This will ensure that upon installation of our package, all of the packages referenced are also installed so that users have those dependent functions as well. Packages can also be added as `Depends:`. However, packages under this section will be attached whenever your package is attached using `library()`. This adjusts the search path and therefore changes the environment and should only be done when necessary. 

In [ ]:
# add stringr to imports list
cat("Imports:\n   stringr\n", file = paste0(pkg_dir, "DESCRIPTION"), append = T)

Now that our function is updated and the necessary package import is added to the DESCRIPTION file we can refresh our package to bring in these new updates and test it out. Remember, we must re-rerun the `doucment()` function to bring in these edits to our `.rd` files, a critical step to incorporating changes into the package infastructure. 

In [ ]:
# document and install 
document(pkg_dir)
install(pkg_dir)

In [ ]:
# re-attach our package
library(basename(pkg_dir), character.only = T)

# this time, TRUE will be printed thanks to our stringr addition
random_number(1, 9)


#### 6. Adding data objects

In order to add data objects, create a new dir `data-raw` and for each data object, create a .R script like done so below. The function `use_data()` will create an .Rda object in a newly created `data/` dir. After doing this, document and install to make the data object part of your package and usable by functions or on its own after package installation. Saving the code for how you created your .Rda file in `data-raw` is actually not required, but it proper semantics and is highly encouraged because it acts as a record to how the data object was created.

In [ ]:
system(paste0("mkdir ", pkg_dir, "data-raw"))

In [ ]:
writeLines("library(devtools)
clr <- \"#EB552C\"
use_data(clr, overwrite = T)",
paste0(pkg_dir, "/data-raw/clr.R"))

In [ ]:
old_dir <- getwd()

# annoyingly need to have wd to the current package... I think. Usually I just run use_data() within the .R script created in data-raw rather than from the cl
setwd(pkg_dir)
system(paste0("Rscript ", pkg_dir, "data-raw/clr.R"), intern = T)

# returning wd to whatever it was set to previously
setwd(old_dir)

In [ ]:
setwd("/Users/sseale/sseale_storage/internal_projects/pkg_dev_tutorial")

We've just added a change and must refresh our installation once again.

In [ ]:
document(pkg_dir)
install(pkg_dir)

In [ ]:
library(basename(pkg_dir), character.only = T)
print(clr)

In [ ]:
# our function and data object are both now included in the package upon installation!
ls(paste0("package:", basename(pkg_dir)))

#### 7. Incorporating c++ 

Many packages rely on c++ to improve the speed of their functions. Below covers how to incorporate c++ code into your own package with the help of the R package `Rcpp`.

* The first step is to place all .cpp files in a dir titled `src/`.
* Next, add this line above the c++ function: `// [[Rcpp::export]]`. This will tell Rcpp what functions to look for to export.
* Run `compileAttributes()` to generate the bindings neccessary for incorporating c++ code into R scripts. New files will then be created in `R/` and `src/`, you do not ever need to touch these newly made files. 
* To the function that calls the c++ script, add the `roxygen2` tag: `#' @useDynLib <package_name>` 
* Lastly, In the DESCRIPTION file, add the line: `LinkingTo: Rcpp`. There is likely an automated way to do this, but I've found it simple enough to manually add it.
* You can now simply reference the c++ function name in a R package function and it will incorporate it, pre-compiled upon installation.

In [ ]:
if (!"Rcpp" %in% installed.packages()) install.packages("Rcpp")
library(Rcpp)

In [ ]:
system(paste0("mkdir ", pkg_dir, "src"))
writeLines("// [[Rcpp::export]]
int add_five(int num) {
return(num + 5);
}",
paste0(pkg_dir, "src/add_five.cpp"))

In [ ]:
compileAttributes(pkg_dir)

Create a function that includes the c++ function just created

In [ ]:
# adding a new function to call c++ function

writeLines(paste0("#' @title Call plus five
#'
#' @description add 5 to a number
#'
#' @param \\code{num} an int to add 5 to
#'
#' @details increase an int by 5
#'
#' @return An int
#'
#' @author Elton John
#'
#' @examples
#' \\dontrun{
#' random_number(5,20)
#' }
#'
#' @useDynLib ", basename(pkg_dir), "
#'
#' @export addFive
addFive <- function(num=NULL) {
    return(add_five(num))
}"),
paste0(pkg_dir, "/R/addFive.R"))

Edit the DESCRIPTION file.

In [ ]:
cat("LinkingTo: Rcpp\n", file = paste0(pkg_dir, "DESCRIPTION"), append = T)

Again, we've just added a change and must refresh our installation. The c++ code just added will be compiled in this step.

In [ ]:
document(pkg_dir)
install(pkg_dir)

Our package now includes our compiled c++ function.

In [ ]:
library(basename(pkg_dir), character.only = T)
stopifnot(addFive(5) == 10)

Have fun and I hope this was helpful to your R workflow! -Spencer